In [1]:
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from tensorflow.keras.callbacks import TensorBoard

import keras
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

import multiprocessing
import numpy as np
import pickle
import time

Using TensorFlow backend.


In [2]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

In [3]:
# Data scaling
out_min = -1
out_max = 1

X_scaled = np.copy(X)

x_min = np.min(X)
x_max = np.max(X)

cnt = 0
for xi in X:
    x_std = (xi - x_min)/(x_max - x_min)
    x_scld = x_std*(out_max - out_min) + out_min
    X_scaled[cnt] = x_scld
    cnt = cnt + 1

print(X_scaled.shape)
print(np.min(X_scaled))
print(np.max(X_scaled))

(414, 100, 100, 2)
-1.0
1.0


In [5]:
def run_training(X_scaled, y, NAME, layer_size, conv_layer, dense_layer):
    model = keras.Sequential()
    
    model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    for l in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
    model.add(Flatten())
    
    for _ in range(dense_layer):
        model.add(Dense(layer_size))
        model.add(Activation('relu'))
    
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.fit(X_scaled, y,
              batch_size=20,
              epochs=10,
              validation_split=0.2,
              callbacks=[tensorboard])


dense_layers = [0, 1, 2]
layer_sizes = [32, 64, 128]
conv_layers = [1, 2, 3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)
            
            p = multiprocessing.Process(target=run_training, args=(X_scaled, y, NAME, layer_size, conv_layer,
                                                                  dense_layer))
            p.start()
            p.join()

1-conv-32-nodes-0-dense-1594011777
Train on 331 samples, validate on 83 samples
Epoch 1/10
331/331 [==============================] - 1s 3ms/step - loss: 0.5134 - acc: 0.7613 - val_loss: 0.3273 - val_acc: 0.8916
Epoch 2/10
331/331 [==============================] - 0s 574us/step - loss: 0.2774 - acc: 0.8973 - val_loss: 0.1626 - val_acc: 0.9639
Epoch 3/10
331/331 [==============================] - 0s 567us/step - loss: 0.1483 - acc: 0.9577 - val_loss: 0.0859 - val_acc: 1.0000
Epoch 4/10
331/331 [==============================] - 0s 560us/step - loss: 0.0765 - acc: 0.9940 - val_loss: 0.0459 - val_acc: 1.0000
Epoch 5/10
331/331 [==============================] - 0s 564us/step - loss: 0.0435 - acc: 0.9970 - val_loss: 0.0288 - val_acc: 1.0000
Epoch 6/10
331/331 [==============================] - 0s 560us/step - loss: 0.0316 - acc: 1.0000 - val_loss: 0.0150 - val_acc: 1.0000
Epoch 7/10
331/331 [==============================] - 0s 580us/step - loss: 0.0198 - acc: 1.0000 - val_loss: 0.0106 - 

Epoch 9/10
331/331 [==============================] - 0s 1ms/step - loss: 0.0040 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 10/10
331/331 [==============================] - 0s 1ms/step - loss: 0.0021 - acc: 1.0000 - val_loss: 0.0042 - val_acc: 1.0000
1-conv-128-nodes-0-dense-1594011805
Train on 331 samples, validate on 83 samples
Epoch 1/10
331/331 [==============================] - 2s 5ms/step - loss: 0.4920 - acc: 0.7523 - val_loss: 0.2226 - val_acc: 0.9157
Epoch 2/10
331/331 [==============================] - 0s 1ms/step - loss: 0.1690 - acc: 0.9728 - val_loss: 0.0765 - val_acc: 1.0000
Epoch 3/10
331/331 [==============================] - 0s 2ms/step - loss: 0.0636 - acc: 0.9940 - val_loss: 0.0315 - val_acc: 1.0000
Epoch 4/10
331/331 [==============================] - 0s 2ms/step - loss: 0.0279 - acc: 1.0000 - val_loss: 0.0159 - val_acc: 1.0000
Epoch 5/10
331/331 [==============================] - 1s 2ms/step - loss: 0.0157 - acc: 1.0000 - val_loss: 0.0102 - val_acc: 1

Epoch 7/10
331/331 [==============================] - 0s 753us/step - loss: 0.0165 - acc: 0.9909 - val_loss: 0.0047 - val_acc: 1.0000
Epoch 8/10
331/331 [==============================] - 0s 739us/step - loss: 0.0127 - acc: 0.9940 - val_loss: 0.0062 - val_acc: 1.0000
Epoch 9/10
331/331 [==============================] - 0s 740us/step - loss: 0.0220 - acc: 0.9970 - val_loss: 0.0106 - val_acc: 0.9880
Epoch 10/10
331/331 [==============================] - 0s 732us/step - loss: 0.0075 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 1.0000
1-conv-64-nodes-1-dense-1594011844
Train on 331 samples, validate on 83 samples
Epoch 1/10
331/331 [==============================] - 2s 5ms/step - loss: 0.6129 - acc: 0.7221 - val_loss: 0.3184 - val_acc: 0.8916
Epoch 2/10
331/331 [==============================] - 1s 2ms/step - loss: 0.2268 - acc: 0.9154 - val_loss: 0.1968 - val_acc: 0.9277
Epoch 3/10
331/331 [==============================] - 1s 2ms/step - loss: 0.0754 - acc: 0.9849 - val_loss: 0.0235 - val

Epoch 5/10
331/331 [==============================] - 1s 2ms/step - loss: 0.1922 - acc: 0.9758 - val_loss: 0.0586 - val_acc: 0.9759
Epoch 6/10
331/331 [==============================] - 1s 2ms/step - loss: 0.0107 - acc: 0.9970 - val_loss: 0.0094 - val_acc: 1.0000
Epoch 7/10
331/331 [==============================] - 1s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0133 - val_acc: 1.0000
Epoch 8/10
331/331 [==============================] - 1s 2ms/step - loss: 7.4850e-04 - acc: 1.0000 - val_loss: 0.0059 - val_acc: 1.0000
Epoch 9/10
331/331 [==============================] - 1s 2ms/step - loss: 5.0557e-04 - acc: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000
Epoch 10/10
331/331 [==============================] - 1s 2ms/step - loss: 4.5073e-04 - acc: 1.0000 - val_loss: 0.0052 - val_acc: 1.0000
1-conv-32-nodes-2-dense-1594011907
Train on 331 samples, validate on 83 samples
Epoch 1/10
331/331 [==============================] - 1s 4ms/step - loss: 0.5382 - acc: 0.7190 - val_loss: 0.2753 -

In [ ]:
mag, ang = cv2.cartToPolar(flow_s[...,0], flow_s[...,1])
hsv = np.zeros_like(img[-1])
hsv[...,1] = 255
hsv[...,0] = ang*180/np.pi/2
hsv[...,2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)